In [ ]:
from rateslib import *

# Constructing Curves from (CC) Zero Rates

A common type of curve definition in quantitative analysis is to construct a `Curve` from continuously compounded zero coupon rates.

There is a one-to-one equivalence relation between discount factors (DFs) and cc zero rates: 

$$
v = exp ( - d \bar{r} )
$$
where $d$ is the day count fraction (DCF) measured between 'today' and the 'maturity' date of the rate, using the ``convention`` associated with the rates.

In **rateslib** a ``Curve`` is defined by DF nodes, so if one wants to construct a ``Curve`` from zero rates either these have to be manually converted to DFs or the ``Solver`` can be used to determine them via calibration.

### Direct conversion

Writing a manual conversion function is not difficult. We just need to use the above formula directly.

In [ ]:
def curve_from_zero_rates(nodes, convention, calendar):
    start = list(nodes.keys())[0]
    nodes_ = {
        **{date: dual_exp(-dcf(start, date, convention=convention) * r/100.0) 
           for (date,r) in list(nodes.items())}
    }
    return Curve(nodes=nodes_, convention=convention, calendar=calendar)

In [ ]:
curve = curve_from_zero_rates(
    {dt(2024, 7, 15): 0.0, dt(2025, 7, 15): 5.00, dt(2026, 7, 15): 4.65},
    convention="act365f",
    calendar="nyc",
)
curve.plot("1d")

In [ ]:
curve.nodes.nodes

If cubic spline interpolation is required this could be included within the ``curve_from_zero_rates`` function using the ``t`` argument from a ``Curve``.

### Using a Solver and `Value` Instruments

The advantage of using a ``Solver`` is that the ``Curve`` can be calibrated directly without a manually written construction function and derivatives and risk sensitivities are automatically obtained. The easiest way to directly specify this is to use the ``Value`` class. 

In [ ]:
curve = Curve(
    {dt(2024, 7, 15): 1.0, dt(2025, 7, 15): 1.0, dt(2026, 7, 15): 1.0},
    convention="act365f", calendar="nyc", id="ccz_curve"
)
solver = Solver(
    curves=[curve],
    instruments=[
        Value(dt(2025, 7, 15), metric="cc_zero_rate", curves=curve),
        Value(dt(2026, 7, 15), metric="cc_zero_rate", curves=curve),
    ],
    s=[5.0, 4.65]  # <- Same rates to observe same derived discount factors
)

In [ ]:
curve.plot("1d")

In [ ]:
curve.nodes.nodes